In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dm4bem

### Introduction

In this first part we will define our working environment. The plan of our house can be found below. The objective of this study case is the nalysis of a cubic building with 5 identical walls & a transparent wall (glass window), air infiltration, and HVAC system controlling the indoor air temperature.

### Design model

The room that we will study is presented in the figure below. Two rooms are separated by a small wall without insulation. The total area of the model is about 9 meters square and the ceiling is 3 meters high. The rooms have each 3 exterior walls and one interior wall. We will consider the house as insulated from the outside .
The interior walls are made of concrete. The room also has a 1 m² window, a 2 m² door and a HVAC device.



<center><img src="https://hub.gke2.mybinder.org/user/cghiaus-dm4bem-v9wqwu31/files/Adobe%20Scan%2024%20avr.%202023%20(1)%20(1).pdf?_xsrf=2%7C8d3810e3%7C68e4a5cde0ead2325dbafcff04581392%7C1682336278"/></center>

> Figure 1. Plan of the building



We will now define boundary conditions wich are important to solve our case.


### Hypothesis

Boundary Conditions:

- Temperature of the exterior air = 0 °C
- Setpoint temperature of the interior air = 18 °C
- Unique temperature on each wall
- Unidirectionnal heat transfer

### Dimension of windows, doors, and walls.

In [3]:
l = 3               # m length of the cubic room
Sg = l**2           # m² surface of the glass wall
Sc = Si = 5 * Sg    # m² surface of concrete & insulation of the 5 walls
Sc1=2*Sg
Sc2=3*Sg

Thermo-physical properties
The thermophysical properties of the air (in SI units) are:

In [7]:
air = {'Density': 1.2,                      # kg/m³
       'Specific heat in': 1000,
        'Specific heat out' : 25}               # J/(kg·K)
# pd.DataFrame.from_dict(air, orient='index', columns=['air'])
pd.DataFrame(air, index=['Air'])

,Density,Specific heat in,Specific heat out
Air,1.2,1000,25


We use concrete, glass and insulation (glass wool) as materials. We take our door with oak wood. 

In [5]:
BETON = {'Conductivity': 1.400,
            'Density': 2400.0,
            'Specific heat': 880,
            'Width': 0.2,
            'Surface': 60}

ISOLANT = {'Conductivity': 0.046,
              'Density': 25.0,
              'Specific heat': 1030,
              'Width': 0.08,
              'Surface':45}

VERRE = {'Conductivity': 1.4,
         'Density': 2500,
         'Specific heat': 1210,
         'Width': 0.04,
         'Surface': 40}

PORTE = {'Conductivity': 0.15,
         'Density': 800,
         'Specific heat': 2000,
         'Width': 0.1,
         'Surface': 4}

wall = pd.DataFrame.from_dict({'Layer_out': ISOLANT,
                               'Layer_in': BETON,
                               'Glass': VERRE,
                               'Door': PORTE},
                              orient='index')
wall

,Conductivity,Density,Specific heat,Width,Surface
Layer_out,0.046,25.0,1030,0.08,45
Layer_in,1.400,2400.0,880,0.20,60
Glass,1.400,2500.0,1210,0.04,40
Door,0.150,800.0,2000,0.10,4


In [10]:
import numpy as np
A= np.zeros((24,17))
A[0, 0] = 1
A[1, 1] = 1
A[1, 0] = -1
A[2, 2] = 1
A[2, 1] = -1
A[3, 3] = 1
A[3, 2] = -1
A[4, 4] = 1
A[4, 3] = -1
A[6, 6] = 1
A[6, 4] = -1
A[5, 5] = 1
A[5, 4] = -1
A[8, 7] = 1
A[9, 5] = 1
A[9, 7] = -1
A[7, 6] = 1
A[7, 5] = -1
A[10, 6] = 1
A[11, 6] = 1
A[12, 6] = 1
A[12, 8] = -1
A[13, 8] = 1
A[13, 9] = -1
A[14, 16] = 1
A[14, 10] = -1
A[15, 10] = 1
A[15, 11] = -1
A[16, 11] = 1
A[16, 12] = -1
A[17, 12] = 1
A[17, 13] = -1
A[18, 13] = 1
A[19, 15] = 1
A[19, 14] = -1
A[20, 14] = 1
A[21, 9] = 1
A[23, 9] = 1
A[23, 16] = -1
A[22, 9] = 1
A[22, 15] = -1

B=np.zeros(24)
B[0]=1
B[8]=1
B[10]=1
B[11]=1
B[18]=1
B[20]=1
B[21]=1

G=np.zeros((24,24))
G[0,0]=air['Specific heat in']*Sc1
G[1,1]=BETON['Width']/(BETON['Conductivity'])*Sc1
G[2,2]=BETON['Width']/(BETON['Conductivity'])*Sc1
G[3,3]=ISOLANT['Width']/(ISOLANT['Conductivity'])*Sc1
G[4,4]=ISOLANT['Width']/(ISOLANT['Conductivity'])*Sc1
G[5,5]=air['Specific heat out']*Sc1
G[6,6]=air['Specific heat out']*Sc1
G[7,7]=air['Specific heat out']*Sg
G[8,8]=air['Specific heat in']*Sg
G[9,9]=VERRE['Width']/(VERRE['Conductivity'])*Sg
G[10,10]=air['Specific heat in']*Sc1
G[11,11]=air['Specific heat out']*Sg
G[12,12]=BETON['Width']/(BETON['Conductivity'])*Sg
G[13,13]=BETON['Width']/(BETON['Conductivity'])*Sg
G[14,14]=ISOLANT['Width']/(ISOLANT['Conductivity'])*Sc2
G[15,15]=ISOLANT['Width']/(ISOLANT['Conductivity'])*Sc2
G[16,16]=BETON['Width']/(BETON['Conductivity'])*Sc2
G[17,17]=BETON['Width']/(BETON['Conductivity'])*Sc2
G[18,18]=air['Specific heat out']*Sc2
G[19,19]=PORTE['Width']/(PORTE['Conductivity'])*PORTE['Surface']
G[20,20]=air['Specific heat in']*Sc2
G[21,21]=air['Specific heat out']*Sc2
G[22,22]=air['Specific heat out']*Sc2
G[23,23]=air['Specific heat out']*Sc2

C=np.zeros((17,17))
C[1,1]=BETON['Density']*BETON['Width']*BETON['Specific heat']*Sc1
C[1,1]=ISOLANT['Density']*ISOLANT['Width']*ISOLANT['Specific heat']*Sc1
C[1,1]=BETON['Density']*BETON['Width']*BETON['Specific heat']*Sg
C[1,1]=BETON['Density']*BETON['Width']*BETON['Specific heat']*Sc2
C[1,1]=ISOLANT['Density']*ISOLANT['Width']*ISOLANT['Specific heat']*Sc2